# SFE RG 고정점 수치 실험 (초안)

이 노트에서는 03장에서 정의한

$$\epsilon_{\text{mass}} = g_B \frac{\mu}{\sqrt{\lambda}}$$

의 값을 단순한 토이 RG 모형에서 어떻게 IR 고정점으로 유도할 수 있는지 질적으로 살펴본다.

- **주의**: 아래 RG 방정식은 실제 SFE 라그랑지언에서 엄밀히 유도한 것이 아니라, 1-loop Yukawa+스칼라 모형의 전형적인 구조를 따르는 **예시 ODE**이다.
- **목표**: 여러 초기값에 대해 $(g_B, \lambda)$가 IR에서 고정점 $(g_B^*, \lambda^*)$로 흐르면, 그때의
  $$\epsilon_{\text{mass}}^* = g_B^* \frac{\mu^*}{\sqrt{\lambda^*}}$$
  가 $\mathcal{O}(0.1 \sim 1)$ 범위의 보편 상수로 수렴하는지 확인하는 것이다.

여기서는 간단히 $\mu$를 고정된 스케일(예: 1 단위)로 두고, $g_B, \lambda$의 흐름만 본 뒤, $\epsilon_{\text{mass}}^* \propto g_B^*/\sqrt{\lambda^*}$ 의 거동을 조사한다.


In [4]:
import numpy as np
import math

# 토이 RG 방정식 설정
# t = ln(mu) (여기서는 단순히 RG "시간"으로 사용)
# beta_gB = a1 * g_B^3 + a2 * g_B * lambda
# beta_lambda = b1 * lambda^2 + b2 * g_B**4
# 계수들은 1/(16 pi^2) 수준의 양의 상수로 놓는다 (전형적인 1-loop 스케일)

a1 = 1.0 / (16 * math.pi**2)
a2 = 0.5 / (16 * math.pi**2)
b1 = 3.0 / (16 * math.pi**2)
b2 = -2.0 / (16 * math.pi**2)  # Yukawa가 quartic을 감소시키는 효과를 흉내


def beta_gB(gB, lam):
    return a1 * gB**3 + a2 * gB * lam


def beta_lambda(gB, lam):
    return b1 * lam**2 + b2 * gB**4


def rg_flow(gB0, lam0, t_min=-5.0, t_max=0.0, n_steps=5000):
    """단순 Euler 적분으로 (gB, lambda)의 RG 흐름을 적분한다.

    t는 UV(큰 mu) -> IR(작은 mu) 방향으로 감소한다고 가정한다.
    """
    ts = np.linspace(t_max, t_min, n_steps)  # t_max에서 t_min으로 (IR 방향)
    gB = gB0
    lam = lam0
    for i in range(len(ts) - 1):
        dt = ts[i+1] - ts[i]
        # Euler step
        dg = beta_gB(gB, lam) * dt
        dl = beta_lambda(gB, lam) * dt
        gB += dg
        lam += dl
    return gB, lam


# 여러 초기값에 대해 흐름 계산
initial_conditions = [
    (0.2, 0.2),
    (0.5, 0.3),
    (0.8, 0.5),
    (1.0, 0.8),
]

results = []
for g0, l0 in initial_conditions:
    g_star, l_star = rg_flow(g0, l0)
    results.append((g0, l0, g_star, l_star))

# IR 끝점에서의 epsilon_mass* ~ gB*/sqrt(lambda*) 계산 (mu*는 1로 고정)
print("IR 고정점 근사에서의 epsilon_mass* = g_B*/sqrt(lambda*) (toy):")
for g0, l0, g_star, l_star in results:
    if l_star <= 0:
        eps_star = float('nan')
    else:
        eps_star = g_star / math.sqrt(l_star)
    print(f"  초기 (g_B0={g0:.2f}, lambda0={l0:.2f}) -> IR (g_B*={g_star:.3f}, lambda*={l_star:.3f}), epsilon*={eps_star:.3f}")


IR 고정점 근사에서의 epsilon_mass* = g_B*/sqrt(lambda*) (toy):
  초기 (g_B0=0.20, lambda0=0.20) -> IR (g_B*=0.199, lambda*=0.196), epsilon*=0.449
  초기 (g_B0=0.50, lambda0=0.30) -> IR (g_B*=0.494, lambda*=0.295), epsilon*=0.908
  초기 (g_B0=0.80, lambda0=0.50) -> IR (g_B*=0.778, lambda*=0.501), epsilon*=1.100
  초기 (g_B0=1.00, lambda0=0.80) -> IR (g_B*=0.958, lambda*=0.797), epsilon*=1.073


In [5]:
import math

# 1차원 토이 RG: x = g_B / sqrt(lambda) 만 보는 단순 모델
# dx/dt = -k (x - x_star)  형태로, 어떤 초기값 x0에서도 x -> x_star 로 수렴하도록 설계
# 주의: 실제 SFE 라그랑지언에서 유도한 식이 아니라, "고정점" 개념을 직관적으로 보여주는 예시 ODE.

x_star = 0.37   # 목표 고정점 값 (SFE에서 나오는 epsilon_mass ~ 0.37를 흉내)
k = 0.7         # 수렴 속도 계수


def flow_x(x0, x_star=x_star, k=k, t_end=10.0, n_steps=2000):
    """단순 1차 ODE dx/dt = -k(x - x_star)를 Euler로 적분.

    - t는 RG "시간"으로 해석 (크게 의미 부여하지 않고, 고정점 수렴만 본다).
    - t=0에서 x(0)=x0, t=t_end에서 x(t_end)를 반환.
    """
    dt = t_end / n_steps
    x = x0
    for _ in range(n_steps):
        dx = -k * (x - x_star) * dt
        x += dx
    return x


initial_x = [0.05, 0.10, 0.20, 0.50, 1.00, 2.00]

print("1차원 토이 RG: dx/dt = -k(x - x_*) with x_* = 0.37")
for x0 in initial_x:
    x_ir = flow_x(x0)
    print(f"  초기 x0 = {x0:.2f}  ->  IR x* ≈ {x_ir:.4f}")


1차원 토이 RG: dx/dt = -k(x - x_*) with x_* = 0.37
  초기 x0 = 0.05  ->  IR x* ≈ 0.3697
  초기 x0 = 0.10  ->  IR x* ≈ 0.3698
  초기 x0 = 0.20  ->  IR x* ≈ 0.3698
  초기 x0 = 0.50  ->  IR x* ≈ 0.3701
  초기 x0 = 1.00  ->  IR x* ≈ 0.3706
  초기 x0 = 2.00  ->  IR x* ≈ 0.3715


In [6]:
import numpy as np
import math

# 2D RG 모형을 조금 더 체계적으로 스캔하는 셀
# - (g_B, lambda) 초기값 격자에 대해 IR 끝점에서 epsilon_mass* = g_B*/sqrt(lambda*)를 계산
# - 여러 베타함수 계수 세트(case A,B,C)에 대해 통계를 비교
# 주의: 여전히 "구조만" 반영한 토이 1-loop 모형이며, 실제 SFE 라그랑지언에서 유도된 것은 아님.


def set_betas(a1_new, a2_new, b1_new, b2_new):
    """전역 베타함수 계수를 교체한다 (Cell 1에서 정의한 a1,a2,b1,b2 사용)."""
    global a1, a2, b1, b2
    a1, a2, b1, b2 = a1_new, a2_new, b1_new, b2_new


def rg_flow(gB0, lam0, t_min=-5.0, t_max=0.0, n_steps=5000):
    """(g_B, lambda)의 RG 흐름을 단순 Euler로 적분 (Cell 1 버전과 동일 기능).

    t_max(초기) -> t_min(IR) 방향으로 진행.
    """
    ts = np.linspace(t_max, t_min, n_steps)
    gB = gB0
    lam = lam0
    for i in range(len(ts) - 1):
        dt = ts[i+1] - ts[i]
        dg = beta_gB(gB, lam) * dt
        dl = beta_lambda(gB, lam) * dt
        gB += dg
        lam += dl
    return gB, lam


def epsilon_star_from_initial(g0, l0):
    """초기값 (g0,l0)에 대해 IR 끝점의 epsilon_mass*와 (g*,lambda*)를 반환."""
    g_star, l_star = rg_flow(g0, l0)
    if l_star <= 0:
        return math.nan, g_star, l_star
    eps_star = g_star / math.sqrt(l_star)
    return eps_star, g_star, l_star


def scan_case(label, a1_new, a2_new, b1_new, b2_new,
              g_range=(0.2, 1.0), l_range=(0.2, 1.0),
              n_g=5, n_l=5):
    """주어진 베타 계수 세트에 대해 초기값 격자를 스캔하고 epsilon* 통계를 출력."""
    set_betas(a1_new, a2_new, b1_new, b2_new)

    g_vals = np.linspace(g_range[0], g_range[1], n_g)
    l_vals = np.linspace(l_range[0], l_range[1], n_l)

    eps_list = []
    for g0 in g_vals:
        for l0 in l_vals:
            eps_star, g_star, l_star = epsilon_star_from_initial(g0, l0)
            # 물리적으로 의미 없는 경우 / 발산한 경우는 제외
            if not math.isfinite(eps_star):
                continue
            if eps_star <= 0 or eps_star > 5.0:
                continue
            eps_list.append(eps_star)

    print(f"==== {label} ====")
    if not eps_list:
        print("  유효한 epsilon*를 얻지 못함 (lambda*<=0 또는 발산).")
        return

    eps_arr = np.array(eps_list)
    print(f"  샘플 수   : {len(eps_arr)}")
    print(f"  epsilon* 최소: {eps_arr.min():.3f}")
    print(f"  epsilon* 최대: {eps_arr.max():.3f}")
    print(f"  epsilon* 평균: {eps_arr.mean():.3f}")


# 케이스 정의: 계수 세트를 몇 가지 바꿔가며 스캔
# 기본 스케일: 1/(16 pi^2) ~ 0.0063
base = 1.0 / (16 * math.pi**2)

cases = [
    ("Case A (원래 값)",  1.0*base, 0.5*base,  3.0*base, -2.0*base),
    ("Case B (Yukawa 더 강한 영향)", 1.0*base, 0.5*base,  1.5*base, -3.5*base),
    ("Case C (quartic 자기 상호작용 더 강함)", 1.0*base, 0.3*base,  5.0*base, -1.5*base),
]

print("2D 토이 RG에서 epsilon_mass* = g_B*/sqrt(lambda*) 분포 스캔 (초기값 격자)")
for label, A1, A2, B1, B2 in cases:
    scan_case(label, A1, A2, B1, B2,
              g_range=(0.2, 1.0), l_range=(0.2, 1.0),
              n_g=5, n_l=5)



2D 토이 RG에서 epsilon_mass* = g_B*/sqrt(lambda*) 분포 스캔 (초기값 격자)
==== Case A (원래 값) ====
  샘플 수   : 25
  epsilon* 최소: 0.206
  epsilon* 최대: 1.917
  epsilon* 평균: 0.836
==== Case B (Yukawa 더 강한 영향) ====
  샘플 수   : 25
  epsilon* 최소: 0.201
  epsilon* 최대: 1.762
  epsilon* 평균: 0.808
==== Case C (quartic 자기 상호작용 더 강함) ====
  샘플 수   : 25
  epsilon* 최소: 0.213
  epsilon* 최대: 1.989
  epsilon* 평균: 0.858


In [7]:
import numpy as np
import math

# 2D RG 토이 모형에 대해 "목표 epsilon_mass* ~ 0.37"에 가까운 베타함수 계수를
# 간단한 랜덤 서치(수치 최적화)로 찾아보는 셀.
# - 파라미터 p = (c1,c2,c3,c4)에 대해
#   a1 = c1 * base, a2 = c2 * base, b1 = c3 * base, b2 = c4 * base 로 정의.
# - loss = <(epsilon* - epsilon_target)^2> + 0.1 * (범위)
#   를 최소화하도록 p를 조정한다.
# 딥러닝 프레임워크는 쓰지 않고, numpy 기반의 간단한 랜덤 탐색/수치해석만 사용.

epsilon_target = 0.37
base = 1.0 / (16 * math.pi**2)


def loss_for_params(p, g_range=(0.2, 1.0), l_range=(0.2, 1.0), n_g=5, n_l=5):
    """주어진 무차원 계수 p에 대해 loss와 epsilon* 샘플들을 계산."""
    c1, c2, c3, c4 = p
    a1_new = c1 * base
    a2_new = c2 * base
    b1_new = c3 * base
    b2_new = c4 * base
    set_betas(a1_new, a2_new, b1_new, b2_new)

    g_vals = np.linspace(g_range[0], g_range[1], n_g)
    l_vals = np.linspace(l_range[0], l_range[1], n_l)

    eps_list = []
    for g0 in g_vals:
        for l0 in l_vals:
            eps_star, g_star, l_star = epsilon_star_from_initial(g0, l0)
            if not math.isfinite(eps_star):
                continue
            if eps_star <= 0 or eps_star > 5.0:
                continue
            eps_list.append(eps_star)

    if not eps_list:
        return 1e9, None

    eps_arr = np.array(eps_list)
    mse = np.mean((eps_arr - epsilon_target) ** 2)
    spread = eps_arr.max() - eps_arr.min()
    # 평균은 목표값에 가깝게, 분산/범위는 너무 크지 않게 만드는 손실함수
    loss = mse + 0.1 * spread
    return loss, eps_arr


# 간단한 랜덤 서치 (hill-climbing)
rng = np.random.default_rng(123)

# 시작점: Case A에 해당하는 (1.0, 0.5, 3.0, -2.0)
p_best = np.array([1.0, 0.5, 3.0, -2.0])
loss_best, eps_best_arr = loss_for_params(p_best)
print("초기 loss =", loss_best)
if eps_best_arr is not None:
    print("  eps* 범위 = (%.3f, %.3f), 평균 = %.3f" % (eps_best_arr.min(), eps_best_arr.max(), eps_best_arr.mean()))

for it in range(120):
    sigma = 0.5 * (0.98 ** it)  # 점점 탐색 반경을 줄여가는 가우시안 탐색
    p_cand = p_best + sigma * rng.standard_normal(4)
    loss_cand, eps_cand_arr = loss_for_params(p_cand)
    if loss_cand < loss_best and eps_cand_arr is not None:
        p_best = p_cand
        loss_best = loss_cand
        eps_best_arr = eps_cand_arr
        print(f"iter {it:3d} 개선: loss={loss_best:.4g}, p={p_best}, eps*범위=({eps_best_arr.min():.3f},{eps_best_arr.max():.3f}), 평균={eps_best_arr.mean():.3f}")

print("\n최종 최적 파라미터 (무차원 c1,c2,c3,c4):", p_best)
print("  a1,a2,b1,b2 =", p_best * base)
if eps_best_arr is not None:
    print("  eps* 범위    = (%.3f, %.3f)" % (eps_best_arr.min(), eps_best_arr.max()))
    print("  eps* 평균    = %.3f" % eps_best_arr.mean())



초기 loss = 0.585189609382556
  eps* 범위 = (0.206, 1.917), 평균 = 0.836
iter   1 개선: loss=0.5773, p=[ 1.45091314  0.78278086  2.68813281 -1.73444341], eps*범위=(0.203,1.921), 평균=0.828
iter   2 개선: loss=0.5558, p=[ 1.29888401  0.6279696   2.73479256 -2.46719519], eps*범위=(0.204,1.853), 평균=0.823
iter   3 개선: loss=0.5521, p=[ 1.85991261  0.31215749  3.20551535 -2.40304302], eps*범위=(0.207,1.843), 평균=0.824
iter   4 개선: loss=0.5437, p=[ 2.56646187  0.0077901   3.06172052 -2.24726927], eps*범위=(0.209,1.833), 평균=0.820
iter   7 개선: loss=0.5212, p=[ 2.73688396  0.01001003  2.90469099 -2.78126726], eps*범위=(0.208,1.777), 평균=0.810
iter   9 개선: loss=0.5038, p=[ 3.09534597  0.74440059  3.31878173 -2.90279493], eps*범위=(0.205,1.750), 평균=0.799
iter  12 개선: loss=0.4897, p=[ 3.99381128  0.46261621  3.33157565 -2.89178932], eps*범위=(0.206,1.724), 평균=0.792
iter  13 개선: loss=0.4819, p=[ 4.00468223  0.48389731  3.14640934 -3.11611604], eps*범위=(0.206,1.705), 평균=0.789
iter  16 개선: loss=0.4801, p=[ 3.48489785  0.97792836 